# Projeto Final

Você fazem parte do time de Data Science e Analytics da Popolishoshop e receberam uma base de dados contendo as infromações sobre a última Black Friday que ocorreu. O time de negócio solicitou para vocês um relatório, onde especificaram algumas informações e também um estudo para que vocês respondessem utilizando as bases fornecidas.

Para esse desafio, vamos trabalhar com o data set [Black Friday](https://www.kaggle.com/mehdidag/black-friday), que reúne dados sobre transações de compras em uma loja de varejo. Esse dataset está quebrado em diferentes arquivos e é sua função entender como cada um se relaciona com o outro.

Vamos utilizá-lo para praticar a exploração utilizando pandas.

Na tabela a seguir podemos ver os nomes das colunas e as descrições dos campos.

| Coluna                 | Descrição                                                 |
|------------------------|-----------------------------------------------------------|
| User_ID                | ID do usuário                                             |
| Product_ID             | ID do produto                                             |
| Gender                 | Sexo do usuário                                           |
| Age                    | Ano em intervalos                                         |
| Occupation             | Ocupação (mascarada)                                      |
| City_Category          | Categoria da cidade (A, B, C)                             |
| StayInCurrentCityYears | Número de anos de permanência na cidade atual             |
| Marital_Status         | Estado civil                                              |
| ProductCategory1       | Categoria do produto (Mascarada)                          |
| ProductCategory2       | Categoria que o produto pode pertencer também (Mascarada) |
| ProductCategory3       | Categoria que o produto pode pertencer também (Mascarada) |
| Purchase               | Valor da compra                                           | 

Todo o código desenvolvido deve ser pensado para ser reutilizado. A avaliação se dará executando todo o notebook com outra tabela, de mesmas colunas. Sendo assim, pensem na qualidade e reprodução do código.

## _Set up_ da análise

Faça a leitura das três bases fornecidas e junte-as em um único DataFrame.

In [59]:
import pandas as pd

# Imprimir todos valores da célula, não somente o último
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [12]:
df_user_info = pd.read_csv('data/user_profile.csv', delimiter='|')
df_user_info.head()

,User_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status
0,1000001,F,0-17,10,A,2,0
1,1000002,M,55+,16,C,4+,0
2,1000003,M,26-35,15,A,3,0
3,1000004,M,46-50,7,B,2,1
4,1000005,M,26-35,20,A,1,1


In [13]:
df_purchase = pd.read_csv('data/purchase.csv')
df_purchase.head()

,User_ID,Product_ID,Purchase
0,1000001,P00069042,8370
1,1000001,P00248942,15200
2,1000001,P00087842,1422
3,1000001,P00085442,1057
4,1000002,P00285442,7969


In [14]:
df_product_info = pd.read_csv('data/product_info.csv', delimiter=';')
df_product_info.head()

,Product_ID,Product_Category_1,Product_Category_2,Product_Category_3
0,P00069042,3,NaN,NaN
1,P00248942,1,6.0,14.0
2,P00087842,12,NaN,NaN
3,P00085442,12,14.0,NaN
4,P00285442,8,NaN,NaN


In [60]:
df_user_info.shape
df_product_info.shape
df_purchase.shape

(5891, 7)

(3623, 4)

(537577, 3)

In [56]:
df = pd.merge(left=df_purchase, right=df_user_info, how='inner', on='User_ID')
df.head()

,User_ID,Product_ID,Purchase,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status
0,1000001,P00069042,8370,F,0-17,10,A,2,0
1,1000001,P00248942,15200,F,0-17,10,A,2,0
2,1000001,P00087842,1422,F,0-17,10,A,2,0
3,1000001,P00085442,1057,F,0-17,10,A,2,0
4,1000001,P00085942,12842,F,0-17,10,A,2,0


In [57]:
df = pd.merge(left=df, right=df_product_info, how='inner', on='Product_ID')
df.head()

,User_ID,Product_ID,Purchase,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000001,P00069042,8370,F,0-17,10,A,2,0,3,NaN,NaN
1,1000017,P00069042,13591,M,51-55,1,C,0,0,3,NaN,NaN
2,1000023,P00069042,11022,M,36-45,0,B,3,1,3,NaN,NaN
3,1000030,P00069042,8036,F,36-45,7,C,3,0,3,NaN,NaN
4,1000073,P00069042,10885,M,18-25,4,A,4+,0,3,NaN,NaN


## Questão 1

Quantas observações e quantas colunas há no dataset completo (todas as bases juntas)? Responda no formato de uma tuple `(n_observacoes, n_colunas)`.

In [68]:
def final_dataset_shape(df: pd.DataFrame) -> tuple:
    '''
'''
    return df.shape


final_dataset_shape(df)

'(n_observacoes, n_colunas): (537577, 12)'

## Questão 2

Há quantas mulheres com idade entre 26 e 35 anos no dataset? Responda como um único escalar.

In [127]:
def woman_filter(df: pd.DataFrame) -> int:
    '''
'''
    gender_filter = (df['Gender'] == 'F')
    age_filter = (df['Age'] == '26-35')
    
    return df[gender_filter & age_filter]['User_ID'].shape[0]

woman_filter(df)

49348

## Questão 3

Quantos usuários únicos há no dataset? Responda como um único escalar.

In [96]:
def unique_users(df: pd.DataFrame) -> int:
    """
    """
    return df['User_ID'].nunique()


unique_users(df)

5891

## Questão 4

Qual porcentagem dos registros (percentual de linhas) possui ao menos um valor null (`None`, `ǸaN` etc)? Responda como um único escalar entre 0 e 1.

In [140]:
def rows_containing_null(df: pd.DataFrame) -> float:
    """
    """
    rows_containing_null = df[df.isna().any(axis=1)].shape[0]
    total_rows = df.shape[0]

    return (rows_containing_null / total_rows)

rows_containing_null(df)



0.6944102891306734

## Questão 5

Quantos valores null existem na variável (coluna) com o maior número de null? Responda como um único escalar.

## Questão 6

Qual o valor mais frequente (sem contar nulls) em `Product_Category_3`? Responda como um único escalar.

## Questão 7

Podemos afirmar que se uma observação é null em `Product_Category_2` ela também o é em `Product_Category_3`? Responda com um bool (`True`, `False`).

## Questão 8

Qual o ID do usuário que mais gastou na Black Friday?

## Questão 9

Qual grupo (homens ou mulheres) mais gastou na Black Friday?

## Questão 10

Faça uma nova tabela com a categoria mais comprada por cada cliente.

Obs: se ele comprou um produto que possuir valores nas três colunas de categorias, então deve-se considerar todas as categorias.

In [ ]:
Categoria_1 é um intervalo de 1 a 12
Categoria_2 é um intervalo de 1 a 12
Categoria_3 é um intervalo de 1 a 12

Categoria_1: 2
Categoria_2: 4
Categoria_3: 1

In [ ]:
       1 2 3 4 5 6 7 8 9 10 11 12
739273 1 1 0 1 0 0 0 0 0 0  0  0

## Questão 11

Normalize a coluna Purchase. A fórmula de normalização é:


$$x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}} $$

## Questão 12
O estado civil influencia no valor gasto e na categoria de produto comprada? Mostre!

Se eu quisesse vender mais produtos da categoria 14, deveria investir em propagandas para qual estado civil?

## Questão 13
Quais as variáveis que mais impactaram no valor da compra? Como você chegou a essa conclusão?